#**Lab 06-1 Softmax Classification Eager**

* Softmax를 사용하여 분류를 진행한다.


##**기본 Library 선언 및 Tensorflow 버전 확인**

In [1]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(0) # for reproducibility
print(tf.__version__)

2.4.0


In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
#dataset을 선언
#dataset = tf.data.Dataset.from_tensor_slides((x_data, y_data))
#dataset = dataset.repeat().batch(2)

nb_classes = 3 # class의 갯수 선언

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [5]:
#Weight and bias setting 
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W)
print(b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.8041286 , -0.11153453, -0.845551  ],
       [ 0.8489614 ,  0.12237627,  1.3350474 ],
       [ 0.619644  , -0.5498301 , -0.6082269 ],
       [ 0.8465021 , -0.56124455, -1.9332876 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 1.698977 , -0.7732199,  0.2774948], dtype=float32)>


In [6]:
#tf.nn.softmax computes softmax activations
#softmax = exp(logits) / reduce_sum(exp(logits), dim)

def hypothesis(X):
  return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[9.6302575e-01 7.8466479e-03 2.9127620e-02]
 [9.9758720e-01 2.1550851e-03 2.5766526e-04]
 [9.9929500e-01 7.0232514e-04 2.5918425e-06]
 [9.9990988e-01 9.0099180e-05 3.6009638e-08]
 [1.0000000e+00 7.1811779e-09 3.7511960e-08]
 [9.9999988e-01 6.6462839e-08 2.0483291e-10]
 [1.0000000e+00 1.1284286e-09 4.1933465e-10]
 [1.0000000e+00 4.1462063e-11 1.2392547e-11]], shape=(8, 3), dtype=float32)


In [7]:
#Softmax onehot test
sample_db = [[8, 2, 1, 4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[5.6504011e-02 9.4316089e-01 3.3507272e-04]], shape=(1, 3), dtype=float32)


In [8]:
def cost_fn(X, Y):
  logits = hypothesis(X)
  cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
  cost_mean = tf.reduce_mean(cost)
  
  return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(8.657011, shape=(), dtype=float32)


In [10]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
  tape.watch(x)
  y = x * x 
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [11]:
def grad_fn(X, Y):
  with tf.GradientTape() as tape:
    loss = cost_fn(X, Y)
    grads = tape.gradient(loss, variables)
    return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 1.4944656 , -0.748172  , -0.74629366],
       [ 1.7403554 , -1.2476699 , -0.49268553],
       [ 2.7441525 , -1.8728913 , -0.87126136],
       [ 2.8693662 , -1.9980729 , -0.8712933 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.74497724, -0.37365073, -0.3713265 ], dtype=float32)>]


In [12]:
def fit(X, Y, epochs=2000, verbose=100):
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

  for i in range(epochs):
    grads = grad_fn(X, Y)
    optimizer.apply_gradients(zip(grads, variables))
    if (i==0) | ((i+1) % verbose ==0):
      print('Loss at epoch %d: %f' %(i+1, cost_fn(X,Y).numpy()))

fit(x_data, y_data)

Loss at epoch 1: 5.304500
Loss at epoch 100: 0.649903
Loss at epoch 200: 0.577835
Loss at epoch 300: 0.526327
Loss at epoch 400: 0.482917
Loss at epoch 500: 0.444007
Loss at epoch 600: 0.407811
Loss at epoch 700: 0.373085
Loss at epoch 800: 0.338753
Loss at epoch 900: 0.303846
Loss at epoch 1000: 0.268018
Loss at epoch 1100: 0.237778
Loss at epoch 1200: 0.224762
Loss at epoch 1300: 0.214213
Loss at epoch 1400: 0.204564
Loss at epoch 1500: 0.195707
Loss at epoch 1600: 0.187552
Loss at epoch 1700: 0.180019
Loss at epoch 1800: 0.173043
Loss at epoch 1900: 0.166565
Loss at epoch 2000: 0.160536


#**Prediction Check**

In [13]:
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis(sample_data)

print(a)
print(tf.argmax(a, 1))

tf.Tensor([[7.0590724e-04 8.1610464e-02 9.1768360e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [14]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1))

tf.Tensor(
[[4.8042748e-06 1.2345861e-03 9.9876058e-01]
 [7.0590765e-04 8.1610546e-02 9.1768360e-01]
 [9.9533413e-09 1.6533180e-01 8.3466816e-01]
 [7.3936086e-08 8.4991693e-01 1.5008298e-01]
 [2.6441887e-01 7.2347391e-01 1.2107314e-02]
 [1.3960831e-01 8.6036599e-01 2.5651570e-05]
 [7.4986422e-01 2.5010037e-01 3.5390043e-05]
 [9.1223985e-01 8.7759495e-02 6.6494380e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


#**Convert as Class**

In [15]:
class sotfmax_classifier(tf.keras.Model):
  def __init__(self, nb_classes):
    super(sotfmax_classifier, self).__init__()
    self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
    self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')

  def softmax_regression(self, X):
    return tf.nn.softmax(tf.matmul(X, self.W) + self.b)

  def cost_fn(self, X, Y):
    logits = self.softmax_regression(X)
    cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))
    return cost

  def grad_fn(self, X, Y):
    with tf.GradientTape() as tape:
      cost = self.cost_fn(x_data, y_data)
      grads = tape.gradient(cost, self.variables)
      return grads

  def fit(self, X, Y, epochs=2000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
      grads = self.grad_fn(X, Y)
      optimizer.apply_gradients(zip(grads, self.variables))
      if (i==0) | ((i+1)%verbose==0):
        print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
    
model = sotfmax_classifier(nb_classes)
model.fit(x_data, y_data)

Loss at epoch 1: 3.452757
Loss at epoch 100: 0.736643
Loss at epoch 200: 0.622215
Loss at epoch 300: 0.561386
Loss at epoch 400: 0.510279
Loss at epoch 500: 0.462750
Loss at epoch 600: 0.416741
Loss at epoch 700: 0.371323
Loss at epoch 800: 0.326144
Loss at epoch 900: 0.281836
Loss at epoch 1000: 0.245034
Loss at epoch 1100: 0.229716
Loss at epoch 1200: 0.218714
Loss at epoch 1300: 0.208689
Loss at epoch 1400: 0.199512
Loss at epoch 1500: 0.191082
Loss at epoch 1600: 0.183311
Loss at epoch 1700: 0.176125
Loss at epoch 1800: 0.169461
Loss at epoch 1900: 0.163266
Loss at epoch 2000: 0.157492
